# Katib 실습
katib을 사용하여 training code `01-covid19-katib-train.py`를 hyper parameter tunning한다

# 사전준비1. Base Image
Host에 ssh 접속하여 아래와 같이 docker build하고 push한다

```sh
cat << EOF > Dockerfile.kf-base
FROM ubuntu:18.04
ARG DEBIAN_FRONTEND=noninteractive

RUN apt-get update && \
    apt-get install -y python3 python3-pip python3-dev ffmpeg libsm6 libxext6 

RUN  pip3 install --upgrade pip && \
     pip install \
        tensorflow \
        sklearn \
        opencv-python \   
        pillow \
        minio
EOF
```

```sh
# docker build script
REGISTRY=registry.kube-system.svc.cluster.local:30000
TAG=$REGISTRY/kf-base:latest

docker build -f Dockerfile.kf-base -t $TAG . && \
docker push $TAG
```

# 사전준비2. s3 repository(Minio)에 training data 적재
- Minio 레파지토리에 model bucet 생성
- Covid19 데이터셋 다운로드
- Covid19 데이터셋을 Minio에 업로드
- 첨부: 00-uploadDataset-covid19.sh

In [ ]:
! ./00-uploadDataset-covid19.sh

- Mino bucket 확인
  - http://<vm외부IP>:32001/minio/dataset/covid19/

# Step 1. Hyperparamer 수행할 로직을 Containerizing

### 방법 1. docker 명령으로 build
**Host에 ssh 접속**하여 `01-covid19-katib-train.py`를 업로드한 후

사전준비에서 마련한 Base Image를 이용하여 아래와 같이 docker build하고 push한다

```sh
# docker build script
REGISTRY=registry.kube-system.svc.cluster.local:30000
TAG=$REGISTRY/covid19-katib-job:latest

cat << EOF > Dockerfile.covid 
FROM $REGISTRY/kf-base:latest
COPY 01-covid19-katib-train.py /app/
CMD ["python3", "/app/01-covid19-katib-train.py"]
EOF

docker build -f Dockerfile.covid -t $TAG . && \
docker push $TAG
```

### 방법 2. fairing으로 build
아래 셀을 실행하여 fairing으로 image build하면 image tag명이난수로 생성된다
(기존 fairing에서 build만 실행하기 실습과 동일한 코드이다)

In [1]:
from kubeflow.fairing.preprocessors.base import BasePreProcessor 
from kubeflow.fairing.builders.append.append import AppendBuilder

PRIVATE_REGISTRY = 'registry.kube-system.svc.cluster.local:30000'

preproc = BasePreProcessor(
    input_files = ['01-covid19-katib-train.py'], 
    #path_prefix = '/myapp', # default: /app
    command = ['python3'],  # default: python
)    

builder = AppendBuilder(
    preprocessor = preproc,    
    base_image = f'{PRIVATE_REGISTRY}/kf-base:latest', # 사전준비에서 마련한 Base Image
    image_name = 'covid19-katib-job',
    registry = PRIVATE_REGISTRY,
    push = True,
)

builder.build()

print(f'Published Docker image with tag: "{builder.image_tag}"')

[W 201208 05:02:39 append:50] Building image using Append builder...
[I 201208 05:02:39 base:107] Creating docker context: /tmp/fairing_context_2hxuo462
[I 201208 05:02:39 docker_creds_:234] Loading Docker credentials for repository 'registry.kube-system.svc.cluster.local:30000/kf-base:latest'
[W 201208 05:02:39 append:54] Image successfully built in 0.06600473900107318s.
[W 201208 05:02:39 append:94] Pushing image registry.kube-system.svc.cluster.local:30000/covid19-katib-job:87E2A7A1...
[I 201208 05:02:39 docker_creds_:234] Loading Docker credentials for repository 'registry.kube-system.svc.cluster.local:30000/covid19-katib-job:87E2A7A1'
[W 201208 05:02:39 append:81] Uploading registry.kube-system.svc.cluster.local:30000/covid19-katib-job:87E2A7A1
[I 201208 05:02:39 docker_session_:280] Layer sha256:4c1d20cdee96111c8acf1858b62655a37ce81ae48648993542b7ac363ac5c0e5 exists, skipping
[I 201208 05:02:39 docker_session_:280] Layer sha256:c8e37668deea784f47c8726d934adc12b8d20a2b1c50b0b0c18c

Published Docker image with tag: "registry.kube-system.svc.cluster.local:30000/covid19-katib-job:87E2A7A1"


# Step 2. Katib 실행
※ 주의: faring으로 containerizing한 경우, fairing이 생성한 image tag명으로 `03-covid19-katib-random.yaml`내용을 수정한다

```yaml
apiVersion: "kubeflow.org/v1alpha3"
kind: Experiment
...
spec:
...
  trialTemplate:
    goTemplate:
...
          spec:
            template:
              spec:
                containers:
                - name: {{.Trial}}
                  image: registry.kube-system.svc.cluster.local:30000/katib-job:latest  # 여기를 수정
```

### 방법 1. kubectl로 apply
Host에 ssh 접속하여 아래와 같이 kubectl apply 실행
```sh
kubectl apply -f 03-covid19-katib-random.yaml
```

### 방법 2. katib UI에서 submit
katib UI의 submit 화면에서 `03-covid19-katib-random.yaml` 내용을 copy & paste하고 submit 버튼 클릭